In [57]:
from random import randint

import pandas as pd
import numpy as np
import json
import gc
import re

### распаковка json, приведение к удобному виду для pandas

In [6]:
with open('/content/train.json', 'r', encoding='utf-8') as f:
  json_data = json.load(f)
len(json_data)

1799

In [9]:
json_data[0]['extracted_part']

{'text': ['Размер обеспечения исполнения контракта 6593.25 Российский рубль'],
 'answer_start': [1279],
 'answer_end': [1343]}

In [10]:
json_data[0]['text'][1279:1343]


'Размер обеспечения исполнения контракта 6593.25 Российский рубль'

In [11]:
lenght = len(json_data)
for i in range(lenght):
  target = json_data[i]['extracted_part']['text'][0]
  if target:
    json_data[i]['target'] = target

  else: 
    json_data[i]['target'] = np.nan

  del json_data[i]['extracted_part']
  
json_data[0]

{'id': 809436509,
 'text': 'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета

In [12]:
with open('train_unpacked.json', 'w') as f:
  json.dump(json_data, f)

In [13]:
del json_data
gc.collect()

63

### откроем в pandas посмотрим на данные

In [14]:
df = pd.read_json('/content/train_unpacked.json')
df.sample(3)

,id,text,label,target
1796,94811748,ДОГОВОР ПОДРЯДА № _____ «___»___________20____...,обеспечение гарантийных обязательств,None
236,906322760,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%
585,37831512,7 Проект договора № ________________ Поставка ...,обеспечение исполнения контракта,Обеспечение исполнения Договора устанавливаетс...


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1799 non-null   int64 
 1   text    1799 non-null   object
 2   label   1799 non-null   object
 3   target  1492 non-null   object
dtypes: int64(1), object(3)
memory usage: 56.3+ KB


Посмотрим поближе на сами тексты

In [138]:
string = randint(0, 1799)
df.loc[string, 'text'], df.loc[string, 'target'] 

('Идентификационный код закупки: 222634500416663450100100050440000244 КОНТРАКТ №____ на поставку продуктов питания город ______________ «___»__________ 20__ год ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ САМАРСКОЙ ОБЛАСТИ \'\'ШКОЛА-ИНТЕРНАТ № 2 ДЛЯ ОБУЧАЮЩИХСЯ С ОГРАНИЧЕННЫМИ ВОЗМОЖНОСТЯМИ ЗДОРОВЬЯ  который является казенным учреждением; 2) осуществления закупки услуги по предоставлению кредита; 3) заключения бюджетным учреждением, государственным, муниципальным унитарными предприятиями контракта, предметом которого является выдача независимой гарантии. Заказчик вправе установить, что настоящий раздел не применяется в случае заключения Контракта по результатам проведения электронного запроса котировок (за исключением случая, предусмотренного подпунктом "б" пункта 2 части 10 статьи 24 Федерального закона «О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд»). По контрактам, подлежащим казначейскому сопровождению,

В текстах не везде расставлены знаки препинания, поэтому мы не можем чётко разделить предложения и можем ориентироваться только на регистр

Сложность составляют слова, которые в юридических документах пишутся с заглавной буквы: Заказчик, Договор, Сторон и т.д.

Попробуем написать функцию для разделения текстов на предложения

In [178]:
def lower_the_unnessesary_words(text):
  # сделаем слова, которые пишутся с заглавной буквы и не стоят в начале предложения прописными
  capital_words_exceptions = [
      'Заказчик', 'Получател', 'Договор', 'Поставщик', 
      'Федер', 'Закон', 'Росси', 'Контракт', 'Гражданск', 
      'Министерств', 'Директор', 'Сторон', 'Оборудован', 'Акт'
  ]
  splitted_text = text.split(' ')
  for i in range(len(splitted_text)):
    for exception in capital_words_exceptions:  
      if type(splitted_text[i]) is str and splitted_text[i].startswith(exception):
        splitted_text[i] = splitted_text[i].lower()

  return ' '.join(splitted_text)
          

def text_separation_by_sentences(text, min_sentence_lenght=20):
  # сначала разделим текст по заглавным и уберём неинформативные предложения
  sentences = []
  text = text.replace('\n', ' ')
  text = text.split('.')
  text = ' '.join(text)

  text = lower_the_unnessesary_words(text)
  raw_sentences = re.split(r'\s+(?=[А-Я])', text)

  for i in raw_sentences:
    if len(i) > min_sentence_lenght:
      sentences.append(i)

  return sentences

In [179]:
string = """
'Идентификационный код закупки: 222634500416663450100100050440000244 
КОНТРАКТ №____ на поставку продуктов питания город ______________ «___»__________ 20__ год 
ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ САМАРСКОЙ ОБЛАСТИ \'\'ШКОЛА-ИНТЕРНАТ № 2 ДЛЯ ОБУЧАЮЩИХСЯ С ОГРАНИЧЕННЫМИ ВОЗМОЖНОСТЯМИ ЗДОРОВЬЯ  который является казенным учреждением; 
2) осуществления закупки услуги по предоставлению кредита; 
3) заключения бюджетным учреждением, государственным, муниципальным унитарными предприятиями контракта, предметом которого является выдача независимой гарантии. 
Заказчик вправе установить, что настоящий раздел не применяется в случае заключения Контракта по результатам проведения электронного запроса котировок (за исключением случая, предусмотренного подпунктом "б" пункта 2 части 10 статьи 24 Федерального закона «О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд»). 
По контрактам, подлежащим казначейскому сопровождению, Заказчик вправе не устанавливать требование обеспечения исполнения контракта. 
В случае установления требования размер обеспечения определяется в соответствии с ч. 6.1 статьи 96 Федерального закона 
«О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд». 
Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта с учетом особенностей, указанных в части 6 статьи 96 и за исключением случаев, предусмотренных частями 6.1. и 6.2 статьи 96 Федерального закона «
О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд».] 
8.1. Обеспечение исполнения настоящего Контракта установлено в размере 365,73, эквивалентном 0,50 % от начальной (максимальной) цены контракта.[footnoteRef:16]. 
[16: В случае заключения контракта по результатам определения поставщиков (подрядчиков, исполнителей) в соответствии с пунктом 1 части 1 статьи 30 Закона № 44-ФЗ 
размер обеспечения исполнения контракта, в том числе предоставляемого с учетом положений статьи 37 Закона № 44-ФЗ, устанавливается от цены, по которой заключается контракт, но не может составлять менее чем размер аванса.] 
Обеспечение исполнения Контракта предоставлено в форме _____________________. 
8.2. Обеспечение исполнения настоящего Контракта обеспечивает все обязательства Поставщика, предусмотренные настоящ Адрес поставки Товара: ________ Подпись: От Заказчика: М.П. (при наличии) От Заказчика: От Поставщика: М.П. (при наличии) М.П. (при наличии)',
 'Размер обеспечение исполнения контракта устанавливается от 0,5 % до 30 % начальной (максимальной) цены контракта
"""
print(*text_separation_by_sentences(string), sep='\n\n')

 'Идентификационный код закупки: 222634500416663450100100050440000244

КОНТРАКТ №____ на поставку продуктов питания город ______________ «___»__________ 20__ год

ОБЛАСТИ ''ШКОЛА-ИНТЕРНАТ № 2

ЗДОРОВЬЯ  который является казенным учреждением;  2) осуществления закупки услуги по предоставлению кредита;  3) заключения бюджетным учреждением, государственным, муниципальным унитарными предприятиями контракта, предметом которого является выдача независимой гарантии   заказчик вправе установить, что настоящий раздел не применяется в случае заключения контракта по результатам проведения электронного запроса котировок (за исключением случая, предусмотренного подпунктом "б" пункта 2 части 10 статьи 24 федерального закона «О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд»)

По контрактам, подлежащим казначейскому сопровождению, заказчик вправе не устанавливать требование обеспечения исполнения контракта

В случае установления требован

Наш разделитель не будет идеально делить предложения, т.к. могут быть предложения, у которых в начале стоит одно из слов-исключений, либо когда мы не внесли слово в список слов-исключений, но оно написано с большой буквы 

### разбиение текстов 

In [183]:
pd.__version__

'1.4.4'

In [184]:
for i in range(len(df)):
  df.at[i, 'text'] = text_separation_by_sentences(df.loc[i, 'text'])

In [186]:
df.sample(4)

,id,text,label,target
1618,427883942,"[Москва «___» __________ 2022 г, Акционерное о...",обеспечение гарантийных обязательств,None
66,747913836,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 5.00%
1728,988947604,"[Приложение № 2 к приказу, Центра» от 01 07 20...",обеспечение гарантийных обязательств,размер банковской гарантии на обеспечение гара...
494,468679705,[Извещение о проведении открытого конкурса в э...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 30.00%


In [187]:
df.to_json('train_splitted.json')